# Index calculus algorithm

We will perform the Index calculus algorithm to compute $\log_{a} b_i$ ($i = 1, 2$) in $\mathbf{Z}_{p}^*$, where $a = 5$, $b_1 = 4389733$, $b_2 = 1234567$, and $p = 9330887$ is a prime number.

In [1]:
from algorithms.euclidean import gcd
from algorithms.factorization import factorizeByBase

p = 9330887
a = 5
b1 = 4389733
b2 = 1234567

We will use a base consisting of $-1$ and all primes less than $50$.

In [2]:
base = [-1] + prime_range(50)

Let us now try to find the logarithms table. We construct a matrix by trying random powers of $a$ and factoring them with the numbers in our base. We stop when the matrix has full rank.

In [3]:
set_random_seed(0)

v = []
s = set()
r, l = 0, len(base)
M = Matrix(nrows=0, ncols=l)
while r < l:
    i = randint(1, p-1)
    if i in s:
        continue
    s.add(i)
    x = pow(a, i, p)
    f = factorizeByBase(Integer(x), base, p)
    if not f:
        continue
    MM = Matrix(list(M) + [f])
    rr = MM.rank()
    if rr > r:
        M = MM
        r = rr
        v.append(i)
print(M)
print(v)

[1 0 2 1 0 0 1 0 0 0 0 0 0 0 2 0]
[1 0 1 0 0 0 0 0 0 0 0 1 3 0 0 0]
[0 3 8 0 1 0 0 0 0 1 0 0 0 0 0 0]
[0 1 2 1 2 0 0 0 0 0 1 0 1 0 0 0]
[1 0 0 0 1 3 1 0 0 0 0 0 0 1 0 0]
[1 4 0 0 1 0 0 1 0 0 1 0 0 0 0 0]
[1 2 0 3 3 0 0 0 0 1 0 0 0 0 0 0]
[1 2 2 0 0 1 1 1 0 1 0 0 0 0 0 0]
[0 8 1 0 0 2 0 0 0 0 0 0 0 0 1 0]
[0 3 1 0 0 1 0 0 1 1 0 0 0 0 0 0]
[0 5 0 1 0 0 0 1 0 0 1 0 1 0 0 0]
[0 0 0 1 1 0 0 0 1 0 0 0 0 2 0 0]
[0 0 0 0 2 0 0 0 1 0 1 0 0 0 0 1]
[0 1 3 0 4 0 1 0 0 0 0 0 0 0 0 0]
[1 4 1 5 0 0 0 0 0 1 0 0 0 0 0 0]
[1 1 1 2 0 2 1 0 1 0 0 0 0 0 0 0]
[1342485, 669118, 7237936, 7922551, 9149713, 4288717, 563606, 4482702, 7606125, 6167680, 7916700, 6825736, 4712055, 7120183, 7905940, 4584527]


We now solve the system of equations we have obtained. The solution represents our table of logarithms and can be used to find logarithms of multiple numbers.

In [4]:
t = M^-1 * Matrix(zip(v)) % (p-1)
t

[4665443]
[6670912]
[2030334]
[      1]
[5786904]
[1078197]
[8534197]
[7606749]
[8519903]
[2519168]
[6200403]
[9068634]
[7409417]
[5590350]
[6037417]
[6410599]

Let us verify that the table is correct.

In [5]:
[pow(a, i, p) for i, in t]

[9330886, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]

We can now find a power of $b_i$ that can be factorized with our base. We will use the following function.

In [6]:
def findPower(b, p, base):
    while True:
        i = randint(1, p-1)
        x = pow(b, i, p)
        f = factorizeByBase(Integer(x), base, p)
        if not f:
            continue
        return (i, f)

Let us find such a power of $b_1$ and its factorization.

In [7]:
i1, f1 = findPower(b1, p, base)
i1, f1

(3157805, [0, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0])

We can now use the logarithm table to obtain a congruence in $\log_a b_1$. However, there may be multiple solutions, and we need to check which one is our answer. The number of solutions is given by the greatest common divisor of $p-1$ and the obtained exponent.

In [8]:
g1 = gcd(i1, p-1)
g1

1

We can thus obtain a solution modulo $p-1$ divided by this GCD.

In [9]:
m1 = ((p-1)/g1)
(x1,), = Matrix([f1])*t / i1 % m1
x1

5753305

Let us now put this into a function which will also verify the potential solutions.

In [10]:
def checkSolutions(a, b, p, t, i, f):
    g = gcd(i, p-1)
    m = (p-1)/g
    (x,), = Matrix([f])*t / i % m
    for j in range(g):
        y = pow(a, x, p)
        print("%d^%d mod %d = %d" % (a, x, p, y))
        if y == b:
            return x
        x += m

checkSolutions(a, b1, p, t, i1, f1)

5^5753305 mod 9330887 = 4389733


5753305

We have thus computed $\log_a b_1$. Let us now compute $\log_a b_2$.

In [11]:
i2, f2 = findPower(b2, p, base)
i2, f2

(5353502, [0, 0, 2, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 0, 0])

In [12]:
checkSolutions(a, b2, p, t, i2, f2)

5^2608331 mod 9330887 = 8096320
5^7273774 mod 9330887 = 1234567


7273774

## Running time comparison

We will now use the function `logarithmTable` to compute tables of logarithms, which will then be used to compute some more discrete logarithms with the function `indexCalculus`. We will measure the evaluation times.

In [13]:
from algorithms.discreteLogarithm import logarithmTable, indexCalculus

aa = 47
bb = 191

In [14]:
pp = 100000000003
table = %time logarithmTable(aa, pp, base, trace = True)
print(table)
%time indexCalculus(aa, bb, pp, base, table, trace = True)

found factorization 47^84497243479 = -1^1 * 2^1 * 3^1 * 5^6 * 7^1 * 11^2 * 23^1 * 29^1 (mod 100000000003)
found factorization 47^55045977185 = 2^1 * 11^1 * 13^3 * 19^2 * 37^1 * 43^1 (mod 100000000003)
found factorization 47^93740896216 = 13^4 * 23^2 * 29^1 (mod 100000000003)
found factorization 47^62252472984 = -1^1 * 2^5 * 3^2 * 5^1 * 7^1 * 11^1 * 17^1 * 19^2 * 41^1 (mod 100000000003)
found factorization 47^98257292990 = -1^1 * 3^1 * 23^4 * 41^1 * 47^1 (mod 100000000003)
found factorization 47^57017421058 = 2^5 * 13^2 * 17^1 * 23^2 (mod 100000000003)
found factorization 47^63831308173 = -1^1 * 2^3 * 5^3 * 7^2 * 19^1 * 31^1 * 41^1 * 43^1 (mod 100000000003)
found factorization 47^67516991484 = -1^1 * 3^1 * 5^1 * 7^4 * 11^2 * 19^1 * 31^2 (mod 100000000003)
found factorization 47^33797337121 = 2^10 * 3^2 * 5^2 * 11^1 * 43^1 * 47^1 (mod 100000000003)
found factorization 47^17498086004 = 2^1 * 7^2 * 11^2 * 13^1 * 29^1 * 31^1 * 43^1 (mod 100000000003)
found factorization 47^31657988253 = 2^1

6935101882

In [15]:
pp = 10000000000000000051
base = [-1] + prime_range(5000)
table = %time logarithmTable(aa, pp, base, trace = True)
%time indexCalculus(aa, bb, pp, base, table, trace = True)

found factorization 47^2179497432067614218 = 2^4 * 5^1 * 107^1 * 293^1 * 317^1 * 643^1 * 2591^1 * 4793^1 (mod 10000000000000000051)
found factorization 47^8204853581526273040 = 2^1 * 3^3 * 7^1 * 19^1 * 101^1 * 109^1 * 1091^1 * 1567^1 * 1889^1 (mod 10000000000000000051)
found factorization 47^5332758726454119214 = 2^4 * 11^1 * 17^1 * 73^1 * 593^1 * 883^1 * 1013^1 * 3889^1 (mod 10000000000000000051)
found factorization 47^5587859396418665063 = 23^1 * 227^2 * 307^1 * 311^1 * 701^1 * 2659^1 (mod 10000000000000000051)
found factorization 47^226077030606921208 = 2^3 * 5^3 * 29^1 * 137^1 * 157^1 * 389^1 * 593^1 * 2957^1 (mod 10000000000000000051)
found factorization 47^999096871500948476 = -1^1 * 2^1 * 379^1 * 677^1 * 821^1 * 1049^1 * 1627^1 * 4159^1 (mod 10000000000000000051)
found factorization 47^2720398736870491859 = -1^1 * 2^1 * 3^3 * 23^1 * 67^1 * 71^1 * 89^2 * 193^1 * 283^1 * 1933^1 (mod 10000000000000000051)
found factorization 47^1271998692131759176 = -1^1 * 2^1 * 3^1 * 17^1 * 61^1 *

found factorization 47^757394578626478646 = -1^1 * 2^1 * 3^1 * 37^2 * 131^1 * 1201^1 * 1291^1 * 1433^1 * 2477^1 (mod 10000000000000000051)
found factorization 47^3670642106027243491 = 3^5 * 5^1 * 11^1 * 59^1 * 89^1 * 113^1 * 809^1 * 1619^1 (mod 10000000000000000051)
found factorization 47^1152506915024311731 = -1^1 * 3^1 * 5^1 * 11^1 * 347^1 * 1637^1 * 3019^1 * 3467^1 * 4339^1 (mod 10000000000000000051)
found factorization 47^843769353555840246 = -1^1 * 3^2 * 23^1 * 31^1 * 83^1 * 89^1 * 149^1 * 571^1 * 719^1 * 1009^1 (mod 10000000000000000051)
found factorization 47^2342528196895492096 = 2^2 * 5^2 * 23^1 * 79^1 * 101^1 * 179^1 * 613^1 * 907^1 * 3767^1 (mod 10000000000000000051)
found factorization 47^325186157213753234 = 2^2 * 3^1 * 47^1 * 257^1 * 1171^1 * 2663^1 * 2963^1 * 3571^1 (mod 10000000000000000051)
found factorization 47^7708485993070530200 = -1^1 * 2^1 * 11^1 * 73^1 * 151^1 * 367^1 * 859^1 * 1039^1 * 2221^1 (mod 10000000000000000051)
found factorization 47^1235282735191417043

found factorization 47^3368485832692685615 = 2^1 * 3^1 * 5^2 * 7^2 * 41^1 * 61^1 * 79^1 * 251^1 * 1571^1 * 3061^1 (mod 10000000000000000051)
found factorization 47^5160252247840033863 = -1^1 * 2^1 * 7^1 * 89^1 * 631^1 * 787^1 * 1373^1 * 2713^1 * 3881^1 (mod 10000000000000000051)
found factorization 47^1138978953907096984 = 2^2 * 3^1 * 29^1 * 47^1 * 269^1 * 337^1 * 353^1 * 3559^1 * 3989^1 (mod 10000000000000000051)
found factorization 47^2855788244171642468 = -1^1 * 2^2 * 3^1 * 5^1 * 7^1 * 47^1 * 431^2 * 617^1 * 1217^1 * 1601^1 (mod 10000000000000000051)
found factorization 47^5086945910651030397 = -1^1 * 3^1 * 89^1 * 127^1 * 281^1 * 311^1 * 401^1 * 947^1 * 1609^1 (mod 10000000000000000051)
found factorization 47^2230405413970961393 = -1^1 * 2^6 * 3^2 * 5^2 * 43^1 * 73^1 * 167^1 * 179^1 * 797^1 * 1667^1 (mod 10000000000000000051)
found factorization 47^8961760904011983737 = -1^1 * 2^2 * 13^1 * 1201^1 * 1627^1 * 1721^1 * 1759^1 * 4967^1 (mod 10000000000000000051)
found factorization 47^1

found factorization 47^2862574341882497157 = 2^1 * 5^1 * 79^1 * 587^1 * 677^1 * 2131^1 * 2803^1 * 4019^1 (mod 10000000000000000051)
found factorization 47^3565829746697688170 = -1^1 * 13^1 * 101^1 * 131^1 * 929^1 * 1279^1 * 1423^1 * 4993^1 (mod 10000000000000000051)
found factorization 47^2396968054994230789 = 2^6 * 3^1 * 7^2 * 19^1 * 73^1 * 173^2 * 593^1 * 1913^1 (mod 10000000000000000051)
found factorization 47^5112545423367201006 = -1^1 * 2^1 * 3^1 * 23^1 * 29^1 * 43^1 * 199^1 * 211^1 * 233^1 * 929^1 * 3701^1 (mod 10000000000000000051)
found factorization 47^1350155702805810727 = 2^1 * 3^2 * 23^1 * 31^1 * 43^1 * 331^1 * 593^1 * 643^1 * 757^1 (mod 10000000000000000051)


KeyboardInterrupt: 

KeyboardInterrupt: 